<h6><center>Big Data Algorithms Techniques & Platforms</center></h6>

<h1>
<hr style=" border:none; height:3px;">
<center>Assignment 1: Introduction to MapReduce</center>
<hr style=" border:none; height:3px;">
</h1>


# Introduction 

For this assignment, as you did in the first lab, do not use the native fuctions map() and reduce() of python.






# Exercise 1 : Bigram count

What is a Bigram? A bigram is a couple of two consecutive words. 

For example, the previous sentence contains the following bigrams: "A bigram", "bigram is", "is a", "a couple", etc.


This exercise can be seen as a simple extension of the word count, take into account this while implementing your data structures and your code.

Counting the bigrams


Work with the Ulysses example, provided in the .zip file for the lab. Use the `extract_ulysses.txt` file to try your code and the `ulysses.txt` file to test your final exercise.

Notice that `ulysses.txt` contains the novel by Irish writer James Joyce and  `extract_ulysses.txt` simply contains a part of it. 








In [16]:
import re

def file_reader(filename):
    words = [line.strip() for line in open(filename)]
    bigrams = []
    for ln in words :
           text = re.sub('\W', ' ', ln.lower())
           words = text.split()
           bigrams.append(list(zip(words, words[1:])))
    return (bigrams)

bigrams = file_reader("./data_1/ulysses_extract.txt")

## Part 1 - unique bigrams
Describe and define a Map Reduce procedure that gives as output the number of unique bigrams that appear in the document.

You can use any data structure you want, here a possible implementation of a function that reads the document and give as output the list of bigram in the text is provided. 

Notice that this function reads the bigrams of an input file and stores them in a list.  Feel free to re-implement your read function
if you want to use a different data-structure.




In [17]:
def map_bigrams(data):
    # Flatten biagram list and join pairs
    bigram_tuples = [' '.join(i) for line in bigrams for i in line]
    return bigram_tuples

def shuffle_bigrams(bigram_tuples):
    # Take unique biagrams
    unique = set(bigram_tuples)
    # Put all similar bigrams together
    list_of_list = [[bigram for bigram in bigram_tuples if bigram == i] for i in unique]
    return list_of_list
                
def reduce_bigrams(list_of_list):
    unique_count = len(list_of_list)
    return unique_count

count_unique_bigrams = reduce_bigrams(shuffle_bigrams(map_bigrams(bigrams)))
print(count_unique_bigrams)

6144


### Adding a combine step

In [18]:
def map(data):
    list_of_list_of_bigrams = [i for i in bigrams if len(i) != 0]
    return list_of_list_of_bigrams

# We add a combine step where we take unique list of bigrams beforer the shuffle step
def combine(list_of_list_of_bigrams): 
    list_of_unique_list_of_bigrams = [list(set(i)) for i in list_of_list_of_bigrams]
    return list_of_unique_list_of_bigrams

def shuffle(list_of_unique_list_of_bigrams):
    list_of_unique_list_of_bigrams = combine(map(bigrams))
    all_bigrams = [' '.join(i) for lists in list_of_unique_list_of_bigrams for i in lists]
    unique = set(all_bigrams)
    list_of_list = [[bigram for bigram in all_bigrams if bigram == i] for i in unique]
    return list_of_list

count_unique_bigrams = reduce_bigrams(shuffle(combine(map(bigrams))))
print(count_unique_bigrams)

6144


## Part 2 - Top five most frequent bigrams

Try to re-use some of the steps you implemented in the previous exercise and provide the functions that lists the top five most frequent bigrams and their counts. Implement an algorithm that uses the Map Reduce approach.



In [19]:
def map_top_bigrams(data):
    bigram_tuples = [' '.join(i) for line in bigrams for i in line]
    return bigram_tuples

def shuffle_top_bigrams(bigram_tuples):
    unique = set(bigram_tuples)
    list_of_list = [[bigram for bigram in bigram_tuples if bigram == i] for i in unique]
    return list_of_list

def reduce_top_bigrams(list_of_list):
    dic = {i[0]: len(i) for i in list_of_list}
    # Sort dictionary in decreasing order
    sorted_dic = {k: v for k, v in sorted(dic.items(), key=lambda x: x[1], reverse=True)}
    # Convert dictionary to list and return first 5 elements
    top_bigrams = list(sorted_dic.items())[0: 5]  
    return top_bigrams

print(reduce_top_bigrams(shuffle_top_bigrams(map_top_bigrams(bigrams))))

[('buck mulligan', 55), ('in the', 38), ('of the', 36), ('he said', 28), ('i m', 25)]


# Exercise 2: Food and statistics

<p align="justify">
<font size="3">
Suppose now that you have a text file generated by a billing system of a supermarket chain. 

Each line represents the list of item in a given bill. For the sake of simplicity we removed the ID of the bill and the unuseful data. In a line you can simply find the date and the list of products that have been bought. The items are separated by a comma. 

A sample line:

`2020-10-9,milk,cheese`

</font>
</p>



## Part 1 - The top products
<p align="justify">
<font size="3">
Describe and implement a Map-Reduce algorithm that, for each year-month, provides the top 10 products (the 10 products that have been sold the most): 

```
2020-01: bread 82, milk 3, ...
2020-02: ...
```


</font>
</p>



In [1]:
supermarket = [i.strip().split(',') for i in open("./data_1/supermarket.txt")]

In [21]:
def map_top_products(data):
    # Split date to retrieve Year-Month as first value of the tuple
    # Second value is the product
    tuples = [('-'.join(i[0].split('-')[0:2]),v) for i in data for v in i if v != i[0]]
    return tuples

def shuffle_top_products(tuples):
    unique = set(tuples)
    # Create a list of unique tuples and join all the similar tuple together
    list_of_list = [[element for element in tuples if element == i] for i in unique]
    return list_of_list

def reduce_top_products(list_of_list):
    dic = {i[0]:len(i) for i in list_of_list}
    unique_month = set([i[0] for i in dic])
    # Creation of a dictionary.
    # The key is the Year-Month. Value is a list with all products sold and the respective quantity sold
    dic_per_month = {i:[(k[1],v) for k,v in dic.items() if k[0] == i] for i in unique_month}
    top_products = {k: sorted(v, key = lambda x: x[1], reverse = True)[:5] for k,v in dic_per_month.items()}
    return top_products

print(reduce_top_products(shuffle_top_products(map_top_products(supermarket))))

{'2015-9': [('tomato', 2), ('thyme', 2), ('rice', 2), ('flour', 2), ('paprika', 2)], '2015-11': [('milk', 5), ('cornstarch', 4), ('peanut', 4), ('celery', 4), ('oregano', 4)], '2015-8': [('cheese', 4), ('lime', 3), ('milk', 3), ('nutmeg', 3), ('pineapple', 3)], '2015-5': [('cornstarch', 5), ('carrot', 5), ('beef', 4), ('cream', 4), ('honey', 4)], '2015-6': [('celery', 3), ('pepper', 3), ('soda', 3), ('broccoli', 2), ('parsley', 2)], '2015-3': [('leaf', 3), ('paprika', 3), ('chili', 2), ('margarine', 2), ('cheese', 2)], '2015-10': [('sugar', 2), ('flour', 2), ('cabbage', 2), ('wine', 2), ('broccoli', 2)], '2015-4': [('eggs', 3), ('onions', 3), ('broccoli', 3), ('chocolate', 2), ('vanilla', 2)], '2015-12': [('wine', 4), ('bread', 4), ('oats', 3), ('oregano', 2), ('pepper', 2)], '2015-7': [('oats', 3), ('parmesan', 3), ('paprika', 3), ('paste', 3), ('leaf', 2)], '2015-1': [('cabbage', 3), ('chocolate', 2), ('almond', 2), ('lemon', 2), ('juice', 2)], '2015-2': [('cucumber', 2), ('yeast', 2


## Part 2 - Income for product

<p align="justify">
<font size="3">
Describe and implement an algorithm that, using the Map-Reduce paradigm, lists for each product the income (for each year/month).
</font>
</p>



In [22]:
prices = [line.strip().split(',') for line in open("./data_1/supermarket_prices.txt")]

In [23]:
def map_income(data,granularity = 2):
    # granularity = 1 if want yearly figures
    # Split date according to granularity
    # Split product-income and convert income into float
    tuples= [('-'.join(i[0].split('-')[:granularity]),v.split('-')[0],float(v.split('-')[1])) for i in data for v in i if v != i[0]]
    # Returns a list of tuples (str(date), str(product name), float(income))
    return tuples
    
def shuffle_income(tuples):
    unique = set([tuple[:2] for tuple in tuples]) # We assume prices can be different
    # Combine similar tuples together
    list_of_list = [[element for element in tuples if element[:2] == i] for i in unique]
    return list_of_list

def reduce_income(list_of_list):
    # This has terrible complexity O(n**3). I assume prices might be different.
    unique = set([element[:2] for ls in list_of_list for element in ls])
    # Creation of ictionary. Key: (str(date), str(product name)). Value: sum(income)
    dic = {}
    for ls in unique:
        for ls in list_of_list:
            income = 0
            for element in ls:
                income += element[2]
            dic[element[:2]] = income 
    return dic

print(reduce_income(shuffle_income(map_income(prices, granularity =2))))
print(reduce_income(shuffle_income(map_income(prices, granularity =1))))

{('2015-5', 'celery'): 4.6, ('2015-9', 'beans'): 6.8, ('2015-2', 'cucumber'): 1.4, ('2015-9', 'spinach'): 12.0, ('2015-2', 'lime'): 1.0, ('2015-4', 'egg'): 6.0, ('2015-12', 'ginger'): 2.0, ('2015-4', 'cucumber'): 1.4, ('2015-2', 'beef'): 15.0, ('2015-6', 'water'): 0.6, ('2015-8', 'water'): 0.6, ('2015-10', 'garlic'): 0.32, ('2015-3', 'soy'): 4.0, ('2015-3', 'milk'): 1.73, ('2015-5', 'egg'): 3.0, ('2015-8', 'peanut'): 4.0, ('2015-6', 'peanut'): 8.0, ('2015-5', 'lime'): 1.0, ('2015-12', 'wine'): 9.0, ('2015-9', 'garlic'): 0.64, ('2015-5', 'broccoli'): 10.0, ('2015-2', 'eggs'): 2.5, ('2015-8', 'tomatoes'): 9.0, ('2015-12', 'celery'): 2.3, ('2015-6', 'tomatoes'): 6.0, ('2015-10', 'flakes'): 6.0, ('2015-3', 'peas'): 2.3, ('2015-10', 'tomato'): 3.2, ('2015-1', 'paprika'): 4.5, ('2015-9', 'sesame'): 6.8, ('2015-2', 'mushrooms'): 14.0, ('2015-3', 'carrots'): 3.0, ('2015-1', 'parsley'): 1.0, ('2015-4', 'corn'): 3.0, ('2015-7', 'onions'): 3.0, ('2015-9', 'flakes'): 6.0, ('2015-5', 'oregano'): 3.


## Part 3 - Average/percentage

<p align="justify">
<font size="3">
Describe and implement a Map-Reduce algorithm that gives, for each pair of products that appear in at least one ticket: the percentage of tickets in wich each product appears, the percentage of tickets in which the products appear together. 

Notice that this exercise shares some similarities with one of the previous exercises. Think how and if you can modify (generalize) one of the functions already implemented before. 

</font>
</p>



In [24]:
import itertools
import numpy as np

def map_percentage(data):
    tickets = [i[1:] for i in supermarket] #remove date
    #create pairs of product using a combination if list greater than 1
    list_of_list_of_tuples = [list(itertools.combinations(i,2)) if len(i)>1 else i for i in tickets] 
    return list_of_list_of_tuples

def shuffle_percentage(list_of_list_of_tuples):
    tickets = [i for i in supermarket]
    unique_pairs = set([tuples for lists in list_of_list_of_tuples for tuples in lists])
    lists_with_simplets = [[i,[prod for products in tickets for prod in products if prod == i[0]],[prod for products in tickets for prod in products if prod == i[1]]] for i in unique_pairs]
    list_with_pairs = [[tup for tuples in list_of_list_of_tuples for tup in tuples if tup == i[0]] for i in lists_with_simplets]

    for idx,v in enumerate(list_with_pairs):
        lists_with_simplets[idx].append(v)

    return lists_with_simplets

def reduce_percentage(lists_with_simplets):
    nb_tickets = len([i for i in supermarket])
    percentages = {i[0]:np.array([len(i[1]),len(i[2]),len(i[3])])/nb_tickets for i in lists_with_simplets}
    return percentages

percentages = reduce_percentage(shuffle_percentage(map_percentage(supermarket)))
percentages

{('chili', 'carrots'): array([0.13, 0.05, 0.02]),
 ('salt', 'zucchini'): array([0.08, 0.07, 0.01]),
 ('cornstarch', 'sesame'): array([0.17, 0.07, 0.01]),
 ('cream', 'beef'): array([0.13, 0.08, 0.01]),
 ('coriander', 'cumin'): array([0.14, 0.06, 0.01]),
 ('cabbage', 'mustard'): array([0.12, 0.15, 0.02]),
 ('seeds', 'peanut'): array([0.06, 0.11, 0.02]),
 ('broccoli', 'raisins'): array([0.14, 0.08, 0.01]),
 ('celery', 'lime'): array([0.15, 0.15, 0.01]),
 ('pepper', 'chili'): array([0.16, 0.13, 0.02]),
 ('cilantro', 'milk'): array([0.09, 0.16, 0.02]),
 ('chili', 'cucumber'): array([0.13, 0.11, 0.02]),
 ('thyme', 'sugar'): array([0.08, 0.12, 0.01]),
 ('parsley', 'vanilla'): array([0.12, 0.14, 0.04]),
 ('eggs', 'rice'): array([0.1 , 0.15, 0.02]),
 ('cinnamon', 'apple'): array([0.14, 0.11, 0.01]),
 ('chicken', 'nuts'): array([0.06, 0.12, 0.02]),
 ('potatoes', 'honey'): array([0.06, 0.09, 0.01]),
 ('spinach', 'soy'): array([0.1 , 0.11, 0.01]),
 ('bread', 'tomatoes'): array([0.12, 0.08, 0.01]),

Look at your code. Can you use a combine operation? 

If you cannot directly think about how you can represent the key-value pairs in a way that allows you to take advantage of a combine.

I honestly don't know how to do this.

In the map step, I think the key must be a ("Year-Month") so that in the combine step we can group by key. However, I can't seem to figure out what the value must be (single product or pair).

In [14]:
import numpy.ma as ma
import pandas as pd
import datetime
from collections import Counter
from itertools import combinations

In [21]:
# I allowed myself to take the date_items, which is simply a numpy array of dimension nxm, where n is the number of items and m is 2, the date (as a string), and the items (all together in one string):
supermarket = np.array([np.array(line.split(',', maxsplit=1)) for line in open("./data_1/supermarket.txt").read().splitlines()])

In [23]:
# The idea here is that the year-month is the key and the receipt items are the value. Of course, this doesn't make sense in this context, but I tried to write the code in a way that would be capable of handling this scenario in a RDD setting with a combine function.

def map_percentage_kvp(date_items):
    # First, I extract the dates from my data and turn the string date into a datetime object, due to ease of accessing months and year values     later on
    dates = list(map(lambda x: x.strftime('%Y-%m'), [datetime.datetime.strptime(value, '%Y-%m-%d') for value in date_items[:, 0]]))
    lambda x: x.strftime('%Y-%m')
    # Then, I split the items into the single items in every receipt, then I split them into every possible combination of 2 items
    single_items = np.array([np.array(item.split(',')) for item in date_items[:, 1]])
    two_items = np.array([np.array(list(combinations(item.split(','), 2))) for item in date_items[:, 1]])
    # I rejoin the items of the bigrams, as I want to have a 1D array as my value, not a 2D array
    two_items_joined = np.array([np.array(list(map(lambda x: ' '.join(x), element))) for element in two_items])
    # As proven above, an item is unique to the receipt. Hence, the if it appears 3 times in the flattened array, then it appears 3% of the        time (as there are 100 receipts in total). This is the concept that I apply here. I create tuples with the unigram or bigram, and the          probability, which is 1/100
    list_single = np.array([np.array(list(zip(items, np.full(len(items), 1/len(date_items))))) for items in single_items])
    list_double = np.array([np.array(list(zip(items, np.full(len(items), 1/len(date_items))))) for items in two_items_joined])
    # I return an array of tuples containing the year-month and the items (bigrams and/or unigrams)
    return np.array([(date, np.append(single, double, 0)) if len(double) > 0 else (date, single) for date, single, double in zip(dates, list_single, list_double)])


# The 'in-code' comments are written from a single RDD worker-node's perspective. The idea of the combine function is to reduce the 'local' (on the same worker node) occurrences for the same unigram/bigram if they belong the corresponding tuples have the same year-month

def combine_percentage_kvp(tuples_date_items):
    # I extract the year-month from the sample of tuples that I received
    unique_dates = np.unique(np.array([date[0] for date in tuples_date_items]))
    # I group the tuples with the same year-month together
    grouped = np.array([np.array([tup for tup in tuples_date_items if tup[0] == uniq]) for uniq in unique_dates])
    # Now, I create a 'local reduce'. For the receipt with the same year-month I add the occurrences for identical items
    new_tuples = []
    for tup in grouped:
        # Get the unique bigrams/unigrams from the grouped (by year-month) arrays
        unique_vals = np.unique(np.concatenate(tup[:, 1])[:, 0])
        # Create arrays of identical bigrams/unigrams
        repeated = [np.concatenate(tup[:, 1])[np.where(np.concatenate(tup[:, 1])[:, 0] == uniq)] for uniq in unique_vals]
        # Create new tuples of unigram/bigram and the sum of occurrences
        new_tuples.append([(np.unique(el[:, 0]), el[:, 1].astype(np.float).sum()) for el in repeated])
    # Finally, return an array of tuples with the year-month and the concatenated (summed) tuples of the bigrams/unigrams
    return np.array(list(zip(unique_dates, np.array(new_tuples))))


# The idea of the shuffle is create an array of arrays of tuples with identical key (year-month)

def shuffle_percentage_kvp(double_tuple):
    # Extract the unique dates from all the tuples
    unique_values = np.unique(double_tuple[:, 0])
    # Group the same year-month tuples with each other
    return np.array([np.array([double_tuple[np.where(double_tuple[:, 0] == unique_value)]]) for unique_value in unique_values])


# The reduce function reduces the dimensions of the array by disregarding the keys and simply looking at the values (tuple of unigram/bigram with te occurrence). Then I create a dictionary so that I can easily extract the value given the key

def reduce_percentage_kvp(list_of_lists):
    # dimensionality reduction
    list_of_lists = list_of_lists[:, 0, 0, :]
    # Extract the unique dates
    unique_dates = np.unique(np.array([item[1] for item in list_of_lists]))
    # Get array of all bigrams/unigrams present
    elements = np.concatenate(np.array([item[1] for item in list_of_lists]))[:, 0]
    # All unique bigrams/unigrams
    unique_elements = np.unique(elements)
    # Get rid of dimensions and create a single array of arrays with tuples that have identical bigrams/unigrams
    items_numbers = np.array([np.array([np.array([item[0], item[1]]) for item in np.concatenate(np.array([item[1] for item in list_of_lists])) if item[0] == uniq]) for uniq in unique_elements])
    # Concatenate into a single array and remove a dimension, so that the array now only has tuples with unique bigrams/unigrams and their         corresponding occurrences
    unique_sum = np.array([(np.unique(el[:, 0]), el[:, 1].sum()) for el in items_numbers])
    # Create a dictionary where the unigram/bigram is the key and the value is the occurrence (sum)
    dict_item_prob = dict(zip(np.concatenate(np.concatenate(unique_sum[:, 0])), unique_sum[:, 1]))
    # Extract all the bigrams from the dictionary keys
    bigrams = [tup.split() for tup in list(dict_item_prob.keys()) if len(tup.split()) > 1]
    # Print in the desired format
    for bigram in bigrams:
        print(f"Together, the two items '{bigram[0]}' and '{bigram[1]}' appear in {round(dict_item_prob[tuple([' '.join(bigram)])[0]]*100, 5)}% of all the receipts.")
        print(f'Alone, "{bigram[0]}" appears in {np.round(dict_item_prob[bigram[0]]*100, 5)}% of all the receipts.')
        print(f'and "{bigram[1]}" appears in {np.round(dict_item_prob[bigram[1]]*100, 5)}% of all the receipts.')
        print('\n')
    
    
reduce_percentage_kvp(shuffle_percentage_kvp(combine_percentage_kvp(map_percentage_kvp(supermarket))))

<ipython-input-23-330b945ea415>:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  single_items = np.array([np.array(item.split(',')) for item in date_items[:, 1]])
<ipython-input-23-330b945ea415>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  two_items = np.array([np.array(list(combinations(item.split(','), 2))) for item in date_items[:, 1]])
<ipython-input-23-330b945ea415>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprec

Together, the two items 'almond' and 'almonds' appear in 2.0% of all the receipts.
Alone, "almond" appears in 11.0% of all the receipts.
and "almonds" appears in 11.0% of all the receipts.


Together, the two items 'almond' and 'beans' appear in 1.0% of all the receipts.
Alone, "almond" appears in 11.0% of all the receipts.
and "beans" appears in 8.0% of all the receipts.


Together, the two items 'almond' and 'bell' appear in 1.0% of all the receipts.
Alone, "almond" appears in 11.0% of all the receipts.
and "bell" appears in 5.0% of all the receipts.


Together, the two items 'almond' and 'broccoli' appear in 1.0% of all the receipts.
Alone, "almond" appears in 11.0% of all the receipts.
and "broccoli" appears in 14.0% of all the receipts.


Together, the two items 'almond' and 'celery' appear in 2.0% of all the receipts.
Alone, "almond" appears in 11.0% of all the receipts.
and "celery" appears in 15.0% of all the receipts.


Together, the two items 'almond' and 'cheese' appear in 1

Alone, "broth" appears in 10.0% of all the receipts.
and "thyme" appears in 8.0% of all the receipts.


Together, the two items 'broth' and 'tomato' appear in 2.0% of all the receipts.
Alone, "broth" appears in 10.0% of all the receipts.
and "tomato" appears in 15.0% of all the receipts.


Together, the two items 'broth' and 'turkey' appear in 1.0% of all the receipts.
Alone, "broth" appears in 10.0% of all the receipts.
and "turkey" appears in 9.0% of all the receipts.


Together, the two items 'broth' and 'vanilla' appear in 2.0% of all the receipts.
Alone, "broth" appears in 10.0% of all the receipts.
and "vanilla" appears in 14.0% of all the receipts.


Together, the two items 'broth' and 'water' appear in 2.0% of all the receipts.
Alone, "broth" appears in 10.0% of all the receipts.
and "water" appears in 12.0% of all the receipts.


Together, the two items 'broth' and 'wheat' appear in 1.0% of all the receipts.
Alone, "broth" appears in 10.0% of all the receipts.
and "wheat" appe

Together, the two items 'chili' and 'wheat' appear in 1.0% of all the receipts.
Alone, "chili" appears in 13.0% of all the receipts.
and "wheat" appears in 6.0% of all the receipts.


Together, the two items 'chili' and 'wine' appear in 1.0% of all the receipts.
Alone, "chili" appears in 13.0% of all the receipts.
and "wine" appears in 14.0% of all the receipts.


Together, the two items 'chili' and 'yeast' appear in 1.0% of all the receipts.
Alone, "chili" appears in 13.0% of all the receipts.
and "yeast" appears in 8.0% of all the receipts.


Together, the two items 'chocolate' and 'parmesan' appear in 1.0% of all the receipts.
Alone, "chocolate" appears in 13.0% of all the receipts.
and "parmesan" appears in 14.0% of all the receipts.


Together, the two items 'cilantro' and 'almond' appear in 2.0% of all the receipts.
Alone, "cilantro" appears in 9.0% of all the receipts.
and "almond" appears in 11.0% of all the receipts.


Together, the two items 'cilantro' and 'almonds' appear in

Together, the two items 'cucumber' and 'parsley' appear in 1.0% of all the receipts.
Alone, "cucumber" appears in 11.0% of all the receipts.
and "parsley" appears in 12.0% of all the receipts.


Together, the two items 'cucumber' and 'peanut' appear in 3.0% of all the receipts.
Alone, "cucumber" appears in 11.0% of all the receipts.
and "peanut" appears in 11.0% of all the receipts.


Together, the two items 'cucumber' and 'peeled' appear in 1.0% of all the receipts.
Alone, "cucumber" appears in 11.0% of all the receipts.
and "peeled" appears in 10.0% of all the receipts.


Together, the two items 'cucumber' and 'pineapple' appear in 1.0% of all the receipts.
Alone, "cucumber" appears in 11.0% of all the receipts.
and "pineapple" appears in 9.0% of all the receipts.


Together, the two items 'cucumber' and 'raisins' appear in 2.0% of all the receipts.
Alone, "cucumber" appears in 11.0% of all the receipts.
and "raisins" appears in 8.0% of all the receipts.


Together, the two items 'cu


Together, the two items 'leaves' and 'juice' appear in 1.0% of all the receipts.
Alone, "leaves" appears in 7.0% of all the receipts.
and "juice" appears in 10.0% of all the receipts.


Together, the two items 'leaves' and 'nutmeg' appear in 1.0% of all the receipts.
Alone, "leaves" appears in 7.0% of all the receipts.
and "nutmeg" appears in 11.0% of all the receipts.


Together, the two items 'leaves' and 'nuts' appear in 1.0% of all the receipts.
Alone, "leaves" appears in 7.0% of all the receipts.
and "nuts" appears in 12.0% of all the receipts.


Together, the two items 'leaves' and 'olive' appear in 2.0% of all the receipts.
Alone, "leaves" appears in 7.0% of all the receipts.
and "olive" appears in 19.0% of all the receipts.


Together, the two items 'leaves' and 'onion' appear in 1.0% of all the receipts.
Alone, "leaves" appears in 7.0% of all the receipts.
and "onion" appears in 13.0% of all the receipts.


Together, the two items 'leaves' and 'oregano' appear in 1.0% of all 



Together, the two items 'oats' and 'peanut' appear in 2.0% of all the receipts.
Alone, "oats" appears in 16.0% of all the receipts.
and "peanut" appears in 11.0% of all the receipts.


Together, the two items 'oats' and 'peas' appear in 2.0% of all the receipts.
Alone, "oats" appears in 16.0% of all the receipts.
and "peas" appears in 9.0% of all the receipts.


Together, the two items 'oats' and 'peeled' appear in 1.0% of all the receipts.
Alone, "oats" appears in 16.0% of all the receipts.
and "peeled" appears in 10.0% of all the receipts.


Together, the two items 'oats' and 'pineapple' appear in 3.0% of all the receipts.
Alone, "oats" appears in 16.0% of all the receipts.
and "pineapple" appears in 9.0% of all the receipts.


Together, the two items 'oats' and 'potatoes' appear in 1.0% of all the receipts.
Alone, "oats" appears in 16.0% of all the receipts.
and "potatoes" appears in 6.0% of all the receipts.


Together, the two items 'oats' and 'raisins' appear in 3.0% of all the

and "mushrooms" appears in 13.0% of all the receipts.


Together, the two items 'paste' and 'nutmeg' appear in 2.0% of all the receipts.
Alone, "paste" appears in 10.0% of all the receipts.
and "nutmeg" appears in 11.0% of all the receipts.


Together, the two items 'paste' and 'nuts' appear in 1.0% of all the receipts.
Alone, "paste" appears in 10.0% of all the receipts.
and "nuts" appears in 12.0% of all the receipts.


Together, the two items 'paste' and 'oil' appear in 2.0% of all the receipts.
Alone, "paste" appears in 10.0% of all the receipts.
and "oil" appears in 9.0% of all the receipts.


Together, the two items 'paste' and 'olive' appear in 2.0% of all the receipts.
Alone, "paste" appears in 10.0% of all the receipts.
and "olive" appears in 19.0% of all the receipts.


Together, the two items 'paste' and 'paprika' appear in 3.0% of all the receipts.
Alone, "paste" appears in 10.0% of all the receipts.
and "paprika" appears in 14.0% of all the receipts.


Together, the two it

and "lemon" appears in 11.0% of all the receipts.


Together, the two items 'seeds' and 'margarine' appear in 1.0% of all the receipts.
Alone, "seeds" appears in 6.0% of all the receipts.
and "margarine" appears in 12.0% of all the receipts.


Together, the two items 'seeds' and 'mushrooms' appear in 1.0% of all the receipts.
Alone, "seeds" appears in 6.0% of all the receipts.
and "mushrooms" appears in 13.0% of all the receipts.


Together, the two items 'seeds' and 'mustard' appear in 1.0% of all the receipts.
Alone, "seeds" appears in 6.0% of all the receipts.
and "mustard" appears in 15.0% of all the receipts.


Together, the two items 'seeds' and 'nutmeg' appear in 1.0% of all the receipts.
Alone, "seeds" appears in 6.0% of all the receipts.
and "nutmeg" appears in 11.0% of all the receipts.


Together, the two items 'seeds' and 'nuts' appear in 1.0% of all the receipts.
Alone, "seeds" appears in 6.0% of all the receipts.
and "nuts" appears in 12.0% of all the receipts.


Together

Alone, "vegetable" appears in 9.0% of all the receipts.
and "lemon" appears in 11.0% of all the receipts.


Together, the two items 'vegetable' and 'margarine' appear in 1.0% of all the receipts.
Alone, "vegetable" appears in 9.0% of all the receipts.
and "margarine" appears in 12.0% of all the receipts.


Together, the two items 'vegetable' and 'mushrooms' appear in 1.0% of all the receipts.
Alone, "vegetable" appears in 9.0% of all the receipts.
and "mushrooms" appears in 13.0% of all the receipts.


Together, the two items 'vegetable' and 'nutmeg' appear in 3.0% of all the receipts.
Alone, "vegetable" appears in 9.0% of all the receipts.
and "nutmeg" appears in 11.0% of all the receipts.


Together, the two items 'vegetable' and 'olive' appear in 2.0% of all the receipts.
Alone, "vegetable" appears in 9.0% of all the receipts.
and "olive" appears in 19.0% of all the receipts.


Together, the two items 'vegetable' and 'parsley' appear in 1.0% of all the receipts.
Alone, "vegetable" a